In [1]:
%conda install matplotlib -y
%pip install tpot
%conda install tpot -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - tpot

Current channels:

  - https://repo.anaconda.com/pkgs/main/osx-arm64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-arm64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate chann

In [2]:
import numpy as np
import pandas as pd

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"

# Assign colum names to the dataset
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'Class']

# Read dataset to pandas dataframe
dataset = pd.read_csv(url, names=names)

dataset.head()

,sepal-length,sepal-width,petal-length,petal-width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
# Preprocessing 
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 4].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=0)

#Feature Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [4]:
print(f"Training X: {X_train}\n Training Y: {y_train}")

Training X: [[ 6.74501145e-01  1.06445364e-01  9.90221459e-01  7.90590793e-01]
 [-5.37177559e-01 -1.24957601e-01  4.21564419e-01  3.96171883e-01]
 [-7.79513300e-01  1.03205722e+00 -1.28440670e+00 -1.31297673e+00]
 [ 3.10997534e-01 -1.24957601e-01  6.49027235e-01  7.90590793e-01]
 [ 6.74501145e-01 -5.87763531e-01  1.04708716e+00  1.31648267e+00]
 [-7.79513300e-01 -8.19166497e-01  8.03701950e-02  2.64698913e-01]
 [-1.73673948e-01  1.72626612e+00 -1.17067529e+00 -1.18150376e+00]
 [ 1.89829664e-01 -8.19166497e-01  7.62758643e-01  5.27644853e-01]
 [ 6.86617933e-02 -1.24957601e-01  2.50967307e-01  3.96171883e-01]
 [-5.25060772e-02 -1.05056946e+00  1.37235899e-01  1.75297293e-03]
 [ 1.15917263e+00 -1.24957601e-01  9.90221459e-01  1.18500970e+00]
 [-1.38535265e+00  3.37848329e-01 -1.39813811e+00 -1.31297673e+00]
 [ 1.28034050e+00  1.06445364e-01  7.62758643e-01  1.44795564e+00]
 [-1.02184904e+00  1.03205722e+00 -1.22754100e+00 -7.87084847e-01]
 [-5.37177559e-01  1.49486315e+00 -1.28440670e+00 

## K-Fold Cross Validation

Using KNN algorithm

In [5]:
### Without K-fold cross validation

# Training and Predictions
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=2)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

# Evaluating the Algorithm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(f"Accuracy measured by normal train-test split: {accuracy_score(y_test, y_pred)*100:.3f}%")

[[11  0  0]
 [ 0 13  0]
 [ 0  1  5]]
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        11
Iris-versicolor       0.93      1.00      0.96        13
 Iris-virginica       1.00      0.83      0.91         6

       accuracy                           0.97        30
      macro avg       0.98      0.94      0.96        30
   weighted avg       0.97      0.97      0.97        30

Accuracy measured by normal train-test split: 96.667%


In [6]:
# Preprocessing 
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 4].values

#Feature Scaling 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)

X = scaler.transform(X)

In [7]:
### With K-fold cross validation

from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits = 4, shuffle = False)

accs = []

for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Training and Predictions
    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors=2)
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)

    # Evaluating the Algorithm
    from sklearn.metrics import confusion_matrix, accuracy_score
    print(confusion_matrix(y_test, y_pred))
    print(f"[Fold no.{fold}]: {accuracy_score(y_test, y_pred)*100:.3f}%")

    accs.append(accuracy_score(y_test, y_pred))

print(f"Average accuracy measured by K-fold cross validation: {sum(accs)/len(accs)*100:.3f}%")

[[13  0  0]
 [ 0 12  0]
 [ 0  2 11]]
[Fold no.0]: 94.737%
[[13  0  0]
 [ 0 10  2]
 [ 0  1 12]]
[Fold no.1]: 92.105%
[[12  0  0]
 [ 0 12  1]
 [ 0  2 10]]
[Fold no.2]: 91.892%
[[12  0  0]
 [ 0 13  0]
 [ 0  2 10]]
[Fold no.3]: 94.595%
Average accuracy measured by K-fold cross validation: 93.332%


## Hyperparameter Tuning

### Grid Search

In [8]:
# Preprocessing 
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 4].values

#Feature Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)

X = scaler.transform(X)

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier()

from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits = 4, shuffle = False)

grid_vals = {'n_neighbors': [1, 2, 3, 4, 5]}
grid_knn = GridSearchCV(estimator=classifier, param_grid=grid_vals, scoring='accuracy', 
                       cv=kfold, refit=True, return_train_score=True) 

# Training and Prediction
grid_knn.fit(X, y)

# Evaluating the Algorithm
display(grid_knn.cv_results_)

{'mean_fit_time': array([0.00020492, 0.00019348, 0.00017303, 0.00015873, 0.00016433]),
 'std_fit_time': array([5.04903062e-05, 2.83149040e-05, 1.49353351e-05, 2.01160157e-06,
        1.06988262e-05]),
 'mean_score_time': array([0.0006606 , 0.0005433 , 0.00051379, 0.00049776, 0.00052059]),
 'std_score_time': array([2.63081607e-04, 3.51052130e-05, 2.76035796e-05, 1.01522301e-05,
        4.37465609e-05]),
 'param_n_neighbors': masked_array(data=[1, 2, 3, 4, 5],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_neighbors': 1},
  {'n_neighbors': 2},
  {'n_neighbors': 3},
  {'n_neighbors': 4},
  {'n_neighbors': 5}],
 'split0_test_score': array([0.94736842, 0.94736842, 0.97368421, 0.97368421, 0.97368421]),
 'split1_test_score': array([0.92105263, 0.92105263, 0.92105263, 0.94736842, 0.92105263]),
 'split2_test_score': array([0.89189189, 0.91891892, 0.89189189, 0.91891892, 0.91891892]),
 'split3_test_score': array([1.    

In [10]:
# Best estimator
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits = 4, shuffle = False)

accs = []

for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    y_pred = grid_knn.best_estimator_.predict(X_test)

    # Evaluating the Algorithm
    from sklearn.metrics import confusion_matrix, accuracy_score
    print(confusion_matrix(y_test, y_pred))
    print(f"[Fold no.{fold}]: {accuracy_score(y_test, y_pred)*100:.3f}%")

    accs.append(accuracy_score(y_test, y_pred))

print(f"Average accuracy measured by K-fold cross validation (best model): {sum(accs)/len(accs)*100:.3f}%")

[[13  0  0]
 [ 0 12  0]
 [ 0  1 12]]
[Fold no.0]: 97.368%
[[13  0  0]
 [ 0 10  2]
 [ 0  1 12]]
[Fold no.1]: 92.105%
[[12  0  0]
 [ 0 12  1]
 [ 0  2 10]]
[Fold no.2]: 91.892%
[[12  0  0]
 [ 0 13  0]
 [ 0  0 12]]
[Fold no.3]: 100.000%
Average accuracy measured by K-fold cross validation (best model): 95.341%


### Random Search

In [11]:
from scipy.stats import norm

from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier()

from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits = 4, shuffle = False)

random_vals = {'p': norm(loc=3, scale=1), 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
random_knn = RandomizedSearchCV(estimator=classifier, param_distributions=random_vals, scoring='accuracy', 
                       cv=kfold, refit=True, return_train_score=True) 

# Training and Prediction
random_knn.fit(X, y)

# Evaluating the Algorithm
display(random_knn.cv_results_)

{'mean_fit_time': array([0.0002225 , 0.00017565, 0.00017864, 0.00017887, 0.00017303,
        0.00016344, 0.00017446, 0.00017715, 0.00018644, 0.00017375]),
 'std_fit_time': array([3.36112595e-05, 8.96589384e-06, 6.45576364e-06, 1.56453961e-05,
        6.50182590e-06, 3.30577484e-06, 8.25626534e-06, 1.65146821e-05,
        2.79036329e-05, 7.98502050e-06]),
 'mean_score_time': array([0.00072598, 0.00063002, 0.0006727 , 0.00066811, 0.00063086,
        0.00060326, 0.00065905, 0.00066304, 0.00069475, 0.00064701]),
 'std_score_time': array([1.57694742e-04, 2.72447348e-05, 1.41256721e-05, 2.53158406e-05,
        1.27514940e-05, 2.48100709e-05, 1.45135780e-05, 3.83527985e-05,
        5.14473375e-05, 1.94332167e-05]),
 'param_n_neighbors': masked_array(data=[1, 3, 8, 7, 4, 1, 9, 5, 7, 7],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_p': masked_array(data=[3.80251232041071

In [12]:
# Best estimator
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits = 4, shuffle = False)

accs = []

for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    y_pred = random_knn.best_estimator_.predict(X_test)

    # Evaluating the Algorithm
    from sklearn.metrics import confusion_matrix, accuracy_score
    print(confusion_matrix(y_test, y_pred))
    print(f"[Fold no.{fold}]: {accuracy_score(y_test, y_pred)*100:.3f}%")

    accs.append(accuracy_score(y_test, y_pred))

print(f"Average accuracy measured by K-fold cross validation (best model): {sum(accs)/len(accs)*100:.3f}%")

[[13  0  0]
 [ 0 12  0]
 [ 0  1 12]]
[Fold no.0]: 97.368%
[[13  0  0]
 [ 0 11  1]
 [ 0  1 12]]
[Fold no.1]: 94.737%
[[12  0  0]
 [ 0 12  1]
 [ 0  1 11]]
[Fold no.2]: 94.595%
[[11  1  0]
 [ 0 13  0]
 [ 0  0 12]]
[Fold no.3]: 97.297%
Average accuracy measured by K-fold cross validation (best model): 95.999%


### Genetic Search

In [13]:
# from tpot import TPOTClassifier
# tpot_clf = TPOTClassifier(generations=10, population_size=50, 
#                           verbosity=2, offspring_size=100, scoring='accuracy', cv=6)

# #Training and prediction
# from sklearn.model_selection import StratifiedKFold
# kfold = StratifiedKFold(n_splits = 4, shuffle = False)

# accs = []

# for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
#     X_train, X_test = X[train_idx], X[test_idx]
#     y_train, y_test = y[train_idx], y[test_idx]

#     tpot_clf.fit(X_train, y_train)
#     y_pred = tpot_clf.predict(X_test)

#     # Evaluating the Algorithm
#     from sklearn.metrics import confusion_matrix, accuracy_score
#     print(confusion_matrix(y_test, y_pred))
#     print(f"[Fold no.{fold}]: {accuracy_score(y_test, y_pred)*100:.3f}%")

#     accs.append(accuracy_score(y_test, y_pred))

# print(f"Average accuracy measured by K-fold cross validation(genetic search): {sum(accs)/len(accs)*100:.3f}%")

## Practice

### K-fold cross validation & hyperparameter tuning Practice
The following is data on cancer patients.  
Classify positive/negative using the KNN algorithm and show that the average accuracy of the model is over 90% through K-fold cross validation (K=6).  
However, use the random search algorithm when determining the hyperparameter.

In [14]:
from scipy.stats import norm
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import datasets

import numpy as np
import pandas as pd

raw = datasets.load_breast_cancer()

example = pd.DataFrame(data=raw.data, columns=raw.feature_names)
example["label"] = raw.target

example.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,label
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [15]:
compact_columns_names = ['worst radius', 'worst texture', 'worst smoothness', 'worst compactness', 'worst symmetry', 'worst fractal dimension']

X = example[compact_columns_names]
y = example["label"]

classifier = KNeighborsClassifier()
kfold = StratifiedKFold(n_splits = 6, shuffle = False)

random_vals = {'p': norm(loc=3, scale=1), 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
random_knn = RandomizedSearchCV(estimator=classifier, param_distributions=random_vals, scoring='accuracy', 
                       cv=kfold, refit=True, return_train_score=True) 

# Training and Prediction
random_knn.fit(X, y)

# Evaluating the Algorithm
display(random_knn.cv_results_)

{'mean_fit_time': array([0.00074029, 0.00063952, 0.00069582, 0.00066376, 0.00065323,
        0.00057848, 0.00065955, 0.00057526, 0.00057916, 0.00054598]),
 'std_fit_time': array([1.60921826e-04, 7.08790095e-05, 1.14192357e-04, 4.80526322e-05,
        1.11796742e-04, 2.02957876e-05, 1.05366327e-04, 4.67623500e-05,
        7.15740909e-05, 1.09845178e-05]),
 'mean_score_time': array([0.00203037, 0.00201146, 0.00189801, 0.00188855, 0.00161719,
        0.00186022, 0.00193202, 0.00170028, 0.00191271, 0.00165335]),
 'std_score_time': array([1.49293760e-04, 7.26699238e-05, 1.12177349e-04, 3.91479669e-05,
        9.51461410e-05, 2.50088349e-05, 1.49846456e-04, 5.61097666e-05,
        2.27066453e-04, 1.77236950e-05]),
 'param_n_neighbors': masked_array(data=[10, 9, 5, 7, 1, 9, 7, 3, 7, 2],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_p': masked_array(data=[2.3285955462851

In [17]:
# Best estimator
kfold = StratifiedKFold(n_splits = 6, shuffle = False)

accs = []

for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
    X_train, X_test = X.loc[train_idx], X.loc[test_idx]
    y_train, y_test = y.loc[train_idx], y.loc[test_idx]

    y_pred = random_knn.best_estimator_.predict(X_test)

    # Evaluating the Algorithm
    print(confusion_matrix(y_test, y_pred))
    print(f"[Fold no.{fold}]: {accuracy_score(y_test, y_pred)*100:.3f}%")

    accs.append(accuracy_score(y_test, y_pred))

print(f"Average accuracy measured by K-fold cross validation (best model): {sum(accs)/len(accs)*100:.3f}%")

[[29  7]
 [ 0 59]]
[Fold no.0]: 92.632%
[[31  5]
 [ 3 56]]
[Fold no.1]: 91.579%
[[32  3]
 [ 0 60]]
[Fold no.2]: 96.842%
[[32  3]
 [ 1 59]]
[Fold no.3]: 95.789%
[[32  3]
 [ 3 57]]
[Fold no.4]: 93.684%
[[34  1]
 [ 3 56]]
[Fold no.5]: 95.745%
Average accuracy measured by K-fold cross validation (best model): 94.378%
